# Single-Supply, Electret Microphone Pre-Amplifier

All information in this document comes from the excellent Texas Instruments Reference Design for a [Single Supply Electret Microphone Pre-Amplifier](http://www.ti.com/lit/ug/tidu765/tidu765.pdf) by John Caldwell

In [1]:
from IPython.display import display, Math, Markdown
from math import *

mega  = 1000000
kilo  = 1000
milli = 1/1000
micro = 1/1000000
nano  = 1/1000000000
pico  = 1/1000000000000

def value(value, unit, decimals=3):
    scale = 1
    if unit[0] == 'p':
        scale = 1000000000000
    elif unit[0] == 'n':
        scale = 1000000000
    elif unit[0] == 'u':
        unit = r'\mu ' + unit[1:] 
        scale = 1000000
    elif unit[0] == 'm':
        scale = 1000
    elif unit[0] == 'k':
        scale = 1/1000
    elif unit[0] == 'M':
        scale = 1/1000000
    elif unit[0] == 'G':
        scale = 1/1000000000
    unit = unit.replace('pi', '\\pi')
    unit = unit.replace('Ohm', '\\Omega')
            
    if '/' in unit:
        nom, denom = unit.split('/', maxsplit=1)
        result = f'{value * scale:.{decimals}f} \\frac{{{nom}}}{{{denom}}}'
    else:
        result = f'{value * scale:.{decimals}f} {unit}'
    
    return result


def show(v, unit, decimals=3, formula=None):
    if formula is not None:
        result = "\\begin{equation*}\n" + formula + ' = '
    else:
        result = '$'
    
    result += value(v, unit, decimals=decimals)
    
    if formula is not None:
        result += "\n\\end{equation*}"
    else:
        result += '$'
    
    display(Math(result))

def fit_e_series(value):
    e24 = [1.0, 1.1, 1.2, 1.3, 1.5, 1.6, 1.8, 2.0, 2.2, 2.4, 2.7, 3.0, 3.3, 3.6, 3.9, 4.3, 4.7, 5.1, 5.6, 6.2, 6.8, 7.5, 8.2, 9.1]
    
    # scale series values to magnitude
    scale = pico
    while value > e24[-1] * scale:
        scale *= 10
        if scale > mega:
            return value
    scaled_e = [e * scale for e in e24]

    for idx, v in enumerate(scaled_e):
        if v > value:
            if idx == 0:
                return scaled_e[idx]
            if scaled_e[idx] - value < value - scaled_e[idx - 1]:
                return scaled_e[idx]
            else:
                return scaled_e[idx - 1]

The basic schematic of the pre-amplifier is shown in Figure 5. To understand its operation consider that the current in the microphone ($I_{MIC}$) has a dc component ($I_{dc}$) necessary to bias the internal JFET, and an AC component ($I_{ac}$) caused by sound waves. If the impedance of capacitor `C3` is much less than `R1` at audio frequencies, then $I_{ac}$ will flow through `C3` and not `R1`. Op amp `U1` acts as a transimpedance amplifier, and attempts to hold its inverting input at a constant voltage ($V_B$) by varying its output. The output voltage of the op amp ($V_{OA}$) will be:

\begin{equation*}
V_{OA} = I_{ac} R_2 + V_B
\end{equation*}

Finally, the DC component of the output signal is removed by capacitor `C5`.

![Schematic](img/schematic_novalues.png)

This topology was selected for a few reasons. First, it allows for single-supply operation to be easily accommodated by biasing the non-inverting input of the op amp to the mid-supply point. Second, the gain of the pre-amp is determined by `R2` but the noise gain of the op amp is determined by the ratio of `R2` to `R1`. Therefore it is possible to achieve lower noise with this topology than with a non-inverting amplifier.

Finally, because capacitor `C3` is chosen to have a very low impedance at audio frequencies, the voltage at the drain of the microphone JFET varies very little, potentially reducing distortion caused by channel length modulation in the JFET

In [2]:
operating_voltage  = 5           # V
max_supply_current = 2.5 * milli # mA

# Calculations

### Gain Calculations

The parameters of the microphone selected for this design are shown in the table below.

In [3]:
mic_sensitivity             = -46           # dBV
mic_operating_voltage       =   2           # Vdc
mic_max_current_consumption =   0.5 * milli # mA
mic_impedance               =   2.2 * kilo  # kOhm
mic_min_signal_to_noise     =  60           # dB

The typical microphone sensitivity will be used to calculate the gain of the amplifier. First, the dB value must be converted to a linear value for these calculations. Microphone sensitivity is given as a dB value relative to $1V$, measured at $94dB$ SPL (1 Pascal). Therefore the sensitivity of the microphone in volts per Pascal of air pressure is:

In [34]:
mic_sensitivity_v_per_pa = pow(10, mic_sensitivity / 20)

show(
    mic_sensitivity_v_per_pa,
    'mV/Pa',
    formula=r'10^{\frac{' + value(mic_sensitivity, 'dB') + '}{20}}'
)

<IPython.core.display.Math object>

However, because the pre-amplifier used in this reference design is a transimpedance type, this must be converted to a value of current per Pascal of air pressure. Most likely, the microphone sensitivity was measured using a $2.2k\Omega$ impedance as indicated in the microphone specification table. The output current per Pascal of air pressure will be:

In [33]:
mic_output_current_per_pa = mic_sensitivity_v_per_pa / mic_impedance

show(
    mic_output_current_per_pa,
    "uA/Pa",
    formula=r'\frac{' + value(mic_sensitivity_v_per_pa, 'mV/Pa') + '}{' + value(mic_impedance, 'kOhm') + '}'
)

<IPython.core.display.Math object>

The gain calculation depends on the maximum sound pressure level expected at the microphone input. For this design, we will use $100dB$ SPL as the maximum sound pressure level expected, and map this sound pressure level to typical line level audio levels ($1.228 V_{rms}$). $100dB$ SPL is an air pressure of $2Pa$, giving a microphone output current of 

In [32]:
v_out = 1.228
mic_max_output_current = mic_output_current_per_pa * 2

show(mic_max_output_current, 'uA', formula='A_{max}')

<IPython.core.display.Math object>

The gain calculation for the transimpedance amplifier is:

\begin{equation*}
V_{out} = I_{Iac} * R_2 \rightarrow R_2 = \frac{V_{OUT}}{I_{IN}}
\end{equation*}

In [31]:
R2 = v_out / mic_max_output_current
show(R2, 'kOhm_{exact}')

R2 = fit_e_series(R2)
show(R2, 'kOhm_{E24}', formula="R_2", decimals=1)

<IPython.core.display.Math object>

<IPython.core.display.Math object>

The feedback capacitor `C2` compensates for parasitic capacitance at the op amp inverting input which can cause instability. Capacitor `C2` also forms a pole with resistor `R2` in the response of the pre-amplifier. The frequency of this pole must be high enough to not affect the microphone transfer function within the audible bandwidth. For this design, a response deviation of $-0.1dB$ at $20kHz$ is acceptable. The location of the pole can be calculated using the relative gain at $20kHz$:

\begin{equation*}
f_p = \frac{f}{\sqrt{(\frac{G_0}{G_f}) ^ 2 - 1}}
\end{equation*}

In [30]:
acceptable_high_freq = 16 * kilo # Hz
acceptable_attenuation = -1.0    # dB

acceptable_attenuation = pow(10, acceptable_attenuation / 20)
freq_pole = acceptable_high_freq / sqrt(pow((1.0 / acceptable_attenuation), 2) - 1)

show(freq_pole, 'kHz', formula='f_p')

<IPython.core.display.Math object>

In the above equation, $G_0$ and $G_f$ are the gains at low frequency and the gain at frequency $f$ respectively. Inserting $20kHz$ for $f$, and $0.989$ ($-0.1dB$) for $G_f$, gives a pole frequency of $f_p$. The feedback capacitor value can then be calculated:

\begin{equation*}
C_2 = \frac{1}{2\pi f_pR_2}
\end{equation*}

In [29]:
C2 = 1 / (2 * pi * freq_pole * R2)
show(C2, 'pF_{(exact)}')

C2 = fit_e_series(C2)
show(C2, 'pF_{(E24)}', decimals=0, formula="C_2")

<IPython.core.display.Math object>

<IPython.core.display.Math object>

### Microphone Bias Resistor and Coupling Capacitor

The internal JFET of the electret microphone is biased by resistor `R1`. The value of this resistor can be calculated from the desired supply voltage ($V_{CC}$), and the microphone operating voltage ($V_{MIC}$) and current consumption ($I_S$) given in the table in the beginning:

\begin{equation*}
R_1 = \frac{V_{CC} - V_{MIC}}{I_S}
\end{equation*}

In [28]:
R1 = (operating_voltage - mic_operating_voltage) / mic_max_current_consumption
show(R1, 'kOhm_{(exact)}')

R1 = fit_e_series(R1)
show(R1, 'kOhm_{(E24)}', decimals=1, formula='R_1')

<IPython.core.display.Math object>

<IPython.core.display.Math object>

A value for `R1` slightly less than the calculated value is used to accommodate variation in the supply voltage.

For this pre-amplifier design, it is beneficial to have the largest value for `R1` possible for two reasons. First, the noise gain of op amp `U1` is:

\begin{equation*}
A_N = 1 + \frac{R_2}{R_1}
\end{equation*}

But the signal gain is directly determined by `R2`. Therefore, increasing the value of `R1` decreases the noise gain of the op amp. Second, capacitor `C3` must be large enough that its impedance is much less than resistor `R1` at audio frequencies. Increasing the value of `R1` allows for smaller capacitances to be used for `C3`.

Resistor `R1` and capacitor `C3` form a high-pass filter. The corner frequency of this filter must be low enough to not attenuate low-frequency sound waves. A $2.5Hz$ corner frequency is used to calculate the value of `C3`:

\begin{equation*}
C_3 = \frac{1}{2 \pi R_1 f_c}
\end{equation*}


In [27]:
lower_corner_freq = 2.5 # Hz

C3 = 1 / (2 * pi * R1 * lower_corner_freq)
show(C3, 'uF_{(exact)}')

C3 = fit_e_series(C3)
show(C3, 'uF_{(E24)}', decimals=1, formula='C_3')

<IPython.core.display.Math object>

<IPython.core.display.Math object>

### Op Amp Bias Network

Resistors `R3` and `R5` center the op amp input and output at the midpoint between the power supplies to allow for the widest possible output signal swing. Therefore $R_3 = R_5$ for $V_B = \frac{V_{CC}}{2}$. $100k\Omega$ resistors were used in order to limit the power supply current drawn by this voltage divider. The current in the voltage divider will be:

\begin{equation*}
I_D = \frac{V_{CC}}{R_3 + R_5}
\end{equation*}


In [35]:
R3 = 100 * kilo # Ohm
R5 = 100 * kilo # Ohm

bias_current = operating_voltage / (R3 + R5)

show(bias_current, 'uA', decimals=0, formula='I_D')

<IPython.core.display.Math object>

Larger resistors may be used if this current needs to be limited further. Capacitor `C6` is included to filter thermal noise created by the resistors and any noise which may be present on the power supply. The corner frequency of the low pass filter formed by `R3`, `R5`, and `C6` is:

\begin{equation*}
f_C = \frac{1}{2 \pi (R_3 \parallel R_5) C_6}
\end{equation*}

This corner frequency should be well below the audible range (<$20Hz$) in order to prevent noise from affecting the audio performance of the design. For simplicity, a the same capacity as `C3` was selected for `C6` as this
value was used elsewhere in the design. The resulting corner frequency is:

In [36]:
C6 = C3

corner_freq = 1 / (2 * pi * 50 * kilo * C6)

show(corner_freq, 'Hz', formula='f_C')

<IPython.core.display.Math object>

### AC Coupling Network

Audio systems are commonly ac-coupled so that only the audio signal is passed between them, and any dc voltages are removed. Ac-coupling is accomplished by capacitor `C5`, but resistors `R4` and `R6` should also be included. Resistor `R6` provides a discharge pathway to prevent the build-up of charge on capacitor `C5`. If charge was allowed to build-up on the capacitor, it could potentially be rapidly discharged when connecting equipment to the output of the pre-amplifier. This rapid discharge can cause audible “thumps” when connecting equipment, and can also damage sensitive circuitry. A $100k\Omega$ resistor is a typically used
for this discharge pathway. 

Resistor `R4` limits the current through the capacitor in case a circuit at a different dc voltage is connected to the pre-amplifier output. This resistor may be 10’s of ohms to several hundred ohms. A value of $49.9\Omega$ is selected to also allow the circuit to also be properly interfaced to $50\Omega$ test equipment.

The ac coupling capacitor, `C5`, forms a high-pass filter with resistor `R6`. However, `R6` will be in parallel with the input impedance of any circuitry attached to the microphone pre-amplifier. For this design we will assume that the combined impedance of `R6` and any attached circuitry will be $10k\Omega$:

\begin{equation*}
R_6 \parallel R_{IN(Ext)} = 10k\Omega
\end{equation*}

The corner frequency should be selected to avoid excessive attenuation at low frequencies. $-0.5dB$ is considered acceptable in this design. The equation further above can be modified in order to determine the corner frequency of a high-pass filter:

\begin{equation*}
f_C = f \sqrt{ (\frac{G_0}{G_f})^2 - 1}
\end{equation*}

In [37]:
R4 = 49.9                     # Ohm
R6 = 100 * kilo               # kOhm
lowest_freq = 50              # Hz
acceptable_attenuation = -0.5 # dB

acceptable_attenuation = pow(10, acceptable_attenuation / 20)
corner_freq = lowest_freq * sqrt(pow(1/acceptable_attenuation, 2) - 1)

show(corner_freq, 'Hz', formula='f_C')

<IPython.core.display.Math object>

`C5` can then be calculated to produce the desired corner frequency:

\begin{equation*}
C_5 = \frac{1}{2 \pi f_C (R_6 \parallel R_{IN(Ext)})}
\end{equation*}

In [38]:
C5 = 1 / (2 * pi * corner_freq * 10 * kilo) 
show(C5, 'uF_{(exact)}')

C5 = fit_e_series(C5)
show(C5, 'uF_{(E24)}', decimals=1, formula='C_5')

<IPython.core.display.Math object>

<IPython.core.display.Math object>

## Component Selection

### Passive Components

1% Thick film resistors in 0603 surface mount packages were used for this design. It should be noted that no additional distortion was observed due to these resistors. Distortion due to resistor non-linearity is typically seen at higher signal levels. Capacitor `C2` must be a NP0/C0G type ceramic capacitor or film capacitor. High-K ceramic capacitors (X7R, Y5V, etc.) will produce distortion if installed at `C2`. Capacitors `C3` and `C5` are tantalum capacitors selected to avoid microphonic behavior associated with high-k ceramics. Because these capacitors are polarized, correct polarity must be observed when they are installed in the circuit. Furthermore, there is minimal ac voltage drop across these capacitors and therefore they do not contribute distortion to the signal path.

The Values for all passive components are as following:

In [16]:
display(Markdown(
'| **Component** | **Value** |\n'
'|:------------- |:---------:|\n'
'| R1 | $' + value(R1, "kOhm", decimals=0) + '$ |\n'
'| C3 | $' + value(C3, "uF",   decimals=1) + '$ |\n'
'| C2 | $' + value(C2, "pF",   decimals=1) + '$ |\n'
'| R2 | $' + value(R2, "kOhm", decimals=0) + '$ |\n'
'| R3 | $' + value(R3, "kOhm", decimals=0) + '$ |\n'
'| R5 | $' + value(R5, "kOhm", decimals=0) + '$ |\n'
'| C6 | $' + value(C6, "uF",   decimals=1) + '$ |\n'
'| R4 | $' + value(R4, "Ohm",  decimals=1) + '$ |\n'
'| C5 | $' + value(C5, "uF",   decimals=0) + '$ |\n'
'| R6 | $' + value(R6, "kOhm", decimals=0) + '$ |\n'
))

| **Component** | **Value** |
|:------------- |:---------:|
| R1 | $6 k\Omega$ |
| C3 | $10.0 \mu F$ |
| C2 | $18.0 pF$ |
| R2 | $270 k\Omega$ |
| R3 | $100 k\Omega$ |
| R5 | $100 k\Omega$ |
| C6 | $10.0 \mu F$ |
| R4 | $49.9 \Omega$ |
| C5 | $1 \mu F$ |
| R6 | $100 k\Omega$ |


### Amplifier

Some basic amplifier selection criteria are given in Table 3. The maximum supply voltage, and power supply current are determined by the design requirements. A SOT23-5 package is preferred in this application to keep the overall solution size small.

In [17]:
display(Markdown(
'| **Criteria** | **Value** |\n'
'|:------------- |:---------:|\n'
'| Max Power Supply Voltage | > $' + value(operating_voltage, "V",   decimals=0) + '$ |\n'
'| Power Supply Current     | < $' + value(max_supply_current, "mA", decimals=1) + '$ |\n'
'| Package                  | SOT23-5 |\n'
'| Amplifiers in Package    | 1 |\n'
))

| **Criteria** | **Value** |
|:------------- |:---------:|
| Max Power Supply Voltage | > $5 V$ |
| Power Supply Current     | < $2.5 mA$ |
| Package                  | SOT23-5 |
| Amplifiers in Package    | 1 |


The required slew rate of the amplifier can be determined by calculating the maximum rate of change for a sine wave at $20kHz$ and line level audio voltages ($1.228 V_{rms} \rightarrow 1.736 V_p$):

\begin{equation*}
R_S = 2 \pi f A
\end{equation*}

In [39]:
slew_rate = 2 * pi * 20 * kilo * 1.736

show(slew_rate * micro, 'V/\mu s', formula="R_S")

<IPython.core.display.Math object>

As a conservative rule, an op amp should be selected with 10 times this slew rate to eliminate any possibility of slew-induced distortion, making the required slew rate: $2.182V/\mu s$.

The op amp selected should not degrade the signal to noise ratio of the microphone itself. To meet this criterion, the output noise spectral densities of the microphone and the amplifier circuit will be compared. Only broadband noise of the op amp and microphone is considered in this analysis. The signal to noise ratio in the microphone parameters table is used to determine the current noise of the microphone. Several assumptions are made about the measurement conditions of this value. It is assumed to be an A-weighted value, measured at $1Pa$ / $94dB$ SPL, with a $2.2k\Omega$ resistor in the JFET drain. In this measurement condition, the signal current of the microphone will be $8.083\mu A_{RMS}$, and the noise current will be:

In [40]:
noise_current = mic_output_current_per_pa / pow(10, (mic_min_signal_to_noise / 20))

show(
    noise_current,
    'nA_{RMS}',
    formula= \
    value(mic_min_signal_to_noise, 'dB', decimals=0) + \
    r' = 20 * log(\frac{I_S}{I_N}) \rightarrow I_N = \frac{' + \
    value(mic_output_current_per_pa, 'uA') + \
    r'}{10^\frac{' + \
    value(mic_min_signal_to_noise, 'dB', decimals=0) + \
    '}{20}}'
)

<IPython.core.display.Math object>

This RMS noise current can be converted to a current noise spectral density by dividing by the square root of bandwidth of integration. An A-weighting curve can be approximated using a $13.5kHz$ noise bandwidth

In [41]:
current_noise_spectral_density = noise_current / sqrt(13.5 * kilo)

show(
    current_noise_spectral_density,
    'pA/\sqrt{Hz}',
    formula=\
    r'\frac{' + \
    value(noise_current, 'nA_{RMS}') + \
    r'}{\sqrt{' + \
    value(13.5 * kilo, 'kHz', decimals=1) + \
    r'}}'
)

<IPython.core.display.Math object>

This current noise spectral density includes contributions from both the microphone and the $2.2k\Omega$ resistor used for the measurement. The current noise contribution of the resistor is:

\begin{equation*}
I_{nR} = \sqrt{\frac{4 K_B T}{R}}
\end{equation*}

In [42]:
from scipy.constants import Boltzmann, convert_temperature

temperature = convert_temperature(25, 'Celsius', 'Kelvin')
current_noise_contribution_of_r = sqrt((4 * Boltzmann * temperature) / mic_impedance)

show(current_noise_contribution_of_r, 'pA/\sqrt{Hz}', formula='I_{nR}')

<IPython.core.display.Math object>

Where T is the absolute temperature in degrees Kelvin and KB is Boltzmann’s constant. Extracting this noise from the total noise value gives the current noise spectral density of the microphone:

\begin{equation*}
I_{nMIC} = \sqrt{I_{nT}^2 - I_{nR}^2}
\end{equation*}

In [43]:
total_noise_current = sqrt(pow(current_noise_spectral_density,2) + pow(current_noise_contribution_of_r, 2))

show(total_noise_current, 'pA/\sqrt{Hz}', formula='I_{nMIC}')

<IPython.core.display.Math object>

The voltage noise spectral density from the microphone at the amplifier output is:

\begin{equation*}
E_{NO(MIC)} = I_{nMIC}R_2
\end{equation*}

In [44]:
voltage_noise_spectral_density = total_noise_current * R2

show(voltage_noise_spectral_density, 'uV/\sqrt{Hz}', formula='E_{NO(MIC)}')

<IPython.core.display.Math object>

To ensure that the signal-to-noise ratio of the microphone is not significantly degraded, we will specify that the output voltage noise of the amplifier should be less than 1/10<sup>th</sup> of the microphone:

\begin{equation*}
E_{NO(AMP)} < \frac{E_{NO(MIC)}}{10}
\end{equation*}

The output noise spectral density of the amplifier circuit is:

\begin{equation*}
E_{NO(AMP)} = A_N \sqrt{E_{NI}^2 + E_{NV}^2 + E_{NR}^2}
\end{equation*}

Where $A_N$ is the noise gain, $E_{NI}$ is the op amp current noise contribution, $E_{NV}$ is the op amp voltage noise contribution, and $E_{NR}$ is the thermal noise contribution from the feedback network. Substituting the microphone noise into the equation, the terms can be rearranged to give some selection criteria for the op amp:

\begin{equation*}
(\frac{E_{NO(MIC)}}{10A_N})^2 - E_{NR}^2 > E_{NI}^2 + E_{NV}^2 \\
(\frac{E_{NO(MIC)}}{10A_N})^2 - E_{NR}^2 > (I_N * R_2 \parallel R_1)^2 + E_{NV}^2
\end{equation*}

The terms to the left of the inequality are already set by the microphone selection and circuit design. The terms to the right of the inequality, op amp input current noise ($I_N$) and input voltage noise ($E_{NV}$), are determined by the op amp selection. Solving for the terms to the left of the inequality first will give the op amp requirements for noise:

\begin{equation*}
A_N = 1 + \frac{R_2}{R_1} \\
E_{NR} = \sqrt{4 K_B T (R_2 \parallel R_1)}
\end{equation*}

In [45]:
noise_gain = 1 + R2 / R1
source_impedance = 1 / (1 / R1 + 1 / R2)

thermal_noise_from_feedback = sqrt(4 * Boltzmann * temperature * source_impedance )
opamp_noise_requirement = pow(voltage_noise_spectral_density / (10 * noise_gain), 2) - pow(thermal_noise_from_feedback, 2) 

show(source_impedance, 'kOhm', formula='R_{SRC}')
show(noise_gain, ' ', formula='A_N')
show(thermal_noise_from_feedback, 'nV/\sqrt{Hz}', formula='E_{NR}')
display(Math(
    '$' + \
    f'{opamp_noise_requirement:.4}' + \
    '> (I_N * ' + f'{source_impedance:.0f}' + ')^2 + E_{NV}^2' + \
    '$'
))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

Due to the high source impedance at the inverting input $R_{SRC}$, the op amp selection will be narrowed to devices with JFET or CMOS input devices which feature extremely low current noise. For this reason the input current noise term can be ignored, allowing a limit of input voltage noise spectral density to be calculated:

In [25]:
display(Math(
    '$' + \
    f'{opamp_noise_requirement:.4}' + \
    r'> E_{NV}^2'
))

input_voltage_noise = sqrt(opamp_noise_requirement)

display(Math(
    '$' + \
    value(input_voltage_noise, 'nV/\sqrt{Hz}') + \
    r'> E_{NV}'
))


<IPython.core.display.Math object>

<IPython.core.display.Math object>

The OP-AMP which should be selected should fit the following requirements:

In [26]:
display(Markdown(
'| **Criteria**                | **Required** |\n'
'|:--------------------------- |:------------:|\n'
'| Max Power Supply Voltage    | > $' + value(operating_voltage, "V") + '$ |\n'
'| Power Supply Current        | < $' + value(max_supply_current, "mA")+ '$ |\n'
'| Package                     | SOT23-5      |\n'
'| Amplifiers in Package       | 1            |\n'
'| Slew Rate                   | $' + value(slew_rate * micro * 10, "V/\mu s") + '$ |\n'
'| Input Voltage Noise         | $' + value(input_voltage_noise, "nV/\sqrt{Hz}") + '$ |\n'
'| Input Current Noise         | N/A          |\n'
))

| **Criteria**                | **Required** |
|:--------------------------- |:------------:|
| Max Power Supply Voltage    | > $5.000 V$ |
| Power Supply Current        | < $2.500 mA$ |
| Package                     | SOT23-5      |
| Amplifiers in Package       | 1            |
| Slew Rate                   | $2.182 \frac{V}{\mu s}$ |
| Input Voltage Noise         | $6.646 \frac{nV}{\sqrt{Hz}}$ |
| Input Current Noise         | N/A          |
